In [1]:
import os

In [12]:
 import ntpath

# function to get filename from path
 def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail 



In [13]:
# get list of anomaly videos
path = ["./Anomaly-Videos-Part-3",
"./Anomaly-Videos-Part-4"]

listOfAbnormal = list()
for path in path:
    for (dirpath, dirnames, filenames) in os.walk(path):
        listOfAbnormal += [os.path.join(dirpath, file) for file in filenames]

# C3D ARCHITECTURE

In [14]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import scipy
from keras.utils.data_utils import get_file
from keras.models import Model

In [15]:
C3D_MEAN_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/c3d_mean.npy'
WEIGHTS_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/sports1M_weights_tf.h5'


def c3d_preprocess_input(video):
    """Preprocess video input to make it suitable for feature extraction.
    The video is resized, cropped, resampled and training mean is substracted
    to make it suitable for the network
    :param video: Video to be processed
    :returns: Preprocessed video
    :rtype: np.ndarray
    """

    intervals = np.ceil(np.linspace(0, video.shape[0] - 1, 16)).astype(int)
    frames = video[intervals]

    # Reshape to 128x171
    reshape_frames = np.zeros((frames.shape[0], 128, 171, frames.shape[3]))
    for i, img in enumerate(frames):
        img = cv2.resize(src=img, dsize=(171,128), interpolation=cv2.INTER_CUBIC)
        reshape_frames[i,:,:,:] = img


    mean_path = get_file('c3d_mean.npy',
                         C3D_MEAN_PATH,
                         cache_subdir='models',
                         md5_hash='08a07d9761e76097985124d9e8b2fe34')

    mean = np.load(mean_path)
    reshape_frames -= mean
    # Crop to 112x112
    reshape_frames = reshape_frames[:, 8:120, 30:142, :]
    # Add extra dimension for samples
    reshape_frames = np.expand_dims(reshape_frames, axis=0)

    return reshape_frames


def C3D(weights='sports1M'):
    """Creation of the full C3D architecture
    :param weights: Weights to be loaded into the network. If None,
    the network is randomly initialized.
    :returns: Network model
    :rtype: keras.model
    """

    if weights not in {'sports1M', None}:
        raise ValueError('weights should be either be sports1M or None')

    if K.image_data_format() == 'channels_last':
        shape = (16, 112, 112, 3)
    else:
        shape = (3, 16, 112, 112)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', name='conv1',
                                            input_shape=shape),
        tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2),
                                            padding='same', name='pool1'),
        
        tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', name='conv2'),
        tf.keras.layers.MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid',
                                            name='pool2'),
        
        tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same',name='conv3a'),
        tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same',name='conv3b'),
        tf.keras.layers.MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2),
                                            padding='valid', name='pool3'),
        
        tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same',name='conv4a'),
        tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same',name='conv4b'),
        tf.keras.layers.MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2),
                                            padding='valid', name='pool4'),
        
        tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same',name='conv5a'),
        tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same',name='conv5b'),
        tf.keras.layers.ZeroPadding3D(padding=(0,1,1)),
        tf.keras.layers.MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2),
                                            padding='valid', name='pool5'),
        
        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(4096, activation='relu', name='fc6'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu', name='fc7'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(487, activation='softmax', name='fc8'),
    ])
    
    if weights == 'sports1M':
        weights_path = get_file('sports1M_weights_tf.h5',
                                WEIGHTS_PATH,
                                cache_subdir='models',
                                md5_hash='b7a93b2f9156ccbebe3ca24b41fc5402')
        model.load_weights(weights_path)
    

    return model


def c3d_feature_extractor():
    """Creation of the feature extraction architecture. This network is
    formed by a subset of the original C3D architecture (from the
    beginning to fc6 layer)
    :returns: Feature extraction model
    :rtype: keras.model
    """
    base_model = C3D(weights='sports1M')
    layer_name = 'fc6'
    #model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc6').output)
    feature_extractor_model = Model(inputs=base_model.input,outputs=base_model.get_layer(layer_name).output)
    return feature_extractor_model

# EXTRACT 3D FEATURES

In [16]:
import sklearn.preprocessing



In [ ]:
#os.mkdir('./raw_normal_train_features')
# os.mkdir('./raw_abnormal_train_features')
# os.mkdir('./processed_normal_train_features')
# os.mkdir('processed_abnormal_train_features')

### PARAMS

In [17]:
frame_height = 240
frame_width = 320
channels = 3

frame_count = 16

features_per_bag = 32

In [18]:
def sliding_window(arr, size, stride):
    """Apply sliding window to an array, getting chunks of
    of specified size using the specified stride
    :param arr: Array to be divided
    :param size: Size of the chunks
    :param stride: Number of frames to skip for the next chunk
    :returns: Tensor with the resulting chunks
    :rtype: np.ndarray
    """
    num_chunks = int((len(arr) - size) / stride) + 2
    result = []
    for i in range(0,  num_chunks * stride, stride):
        if len(arr[i:i + size]) > 0:
            result.append(arr[i:i + size])
    return np.array(result)


def interpolate(features, features_per_bag):
    """Transform a bag with an arbitrary number of features into a bag
    with a fixed amount, using interpolation of consecutive features
    :param features: Bag of features to pad
    :param features_per_bag: Number of features to obtain
    :returns: Interpolated features
    :rtype: np.ndarray
    """
    feature_size = np.array(features).shape[1]
    interpolated_features = np.zeros((features_per_bag, feature_size))
    interpolation_indices = np.round(np.linspace(0, len(features) - 1, num=features_per_bag + 1))
    count = 0
    for index in range(0, len(interpolation_indices)-1):
        start = int(interpolation_indices[index])
        end = int(interpolation_indices[index + 1])

        assert end >= start

        if start == end:
            temp_vect = features[start, :]
        else:
            temp_vect = np.mean(features[start:end+1, :], axis=0)

        temp_vect = temp_vect / np.linalg.norm(temp_vect)

        if np.linalg.norm(temp_vect) == 0:
            print("Error")

        interpolated_features[count,:]=temp_vect
        count = count + 1

    return np.array(interpolated_features)


def extrapolate(outputs, num_frames):
    """Expand output to match the video length
    :param outputs: Array of predicted outputs
    :param num_frames: Expected size of the output array
    :returns: Array of output size
    :rtype: np.ndarray
    """

    extrapolated_outputs = []
    extrapolation_indices = np.round(np.linspace(0, len(outputs) - 1, num=num_frames))
    for index in extrapolation_indices:
        extrapolated_outputs.append(outputs[int(index)])
    return np.array(extrapolated_outputs)





In [ ]:
# !sudo apt-get update

In [ ]:
# !sudo apt-get install ffmpeg libsm6 libxext6  -y

In [19]:
import cv2

def get_video_frames(video_path):
    """Reads the video given a file path
    :param video_path: Path to the video
    :returns: Video as an array of frames
    :rtype: np.ndarray
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        else:
            break
    cap.release()
    return frames

def get_video_clips(video_path):
    """Divides the input video into non-overlapping clips
    :param video_path: Path to the video
    :returns: Array with the fragments of video
    :rtype: np.ndarray
    """
    frames = get_video_frames(video_path)
    clips = sliding_window(frames, frame_count, frame_count)
    return clips, len(frames)


In [ ]:
#os.mkdir('./abnormal_train_features')

In [10]:
#normal_videos_path = "/home/jupyter/project/Training-Normal-Videos-Part-2"
feature_extractor = c3d_feature_extractor()
#normal_videos = os.listdir(normal_videos_path)
#normal_videos.sort()

tf.debugging.set_log_device_placement(True)
#print("Processing normal videos...")
#count = 1
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)  
# for vid_name in normal_videos:
#     if count > 367: 
#         print(count,"/",len(normal_videos), "video:",vid_name)
#         vid_path = os.path.join(normal_videos_path, vid_name)
#         feats_path = os.path.join(
#     './raw_normal_train_features', vid_name[:-9] + ".npy")

#         clips, frames = get_video_clips(vid_path)

#                 # Remove last clip if number of frames is not equal to 16
#         if frames % 16 != 0:
#             clips = clips[:-1]
        
#         prep_clips = [c3d_preprocess_input(np.array(clip)) for clip in clips]
#         prep_clips = np.vstack(prep_clips)

#         features = feature_extractor.predict(prep_clips)
#         features = sklearn.preprocessing.normalize(features, axis=1)

#         with open(feats_path, "wb") as f:
#             np.save(f, features)
    
#     count += 1
  
count = 1
print('Processing Abnormal videos...')
listOfAbnormal.sort()
for vid_path in listOfAbnormal:
    if count > 361:
        vid_name = path_leaf(vid_path)
        print(count,"/",len(listOfAbnormal), "video:", vid_name)
        clips, frames = get_video_clips(vid_path)
        #rint("Processing {}".format(vid_name))
        feats_path = os.path.join(
                    './raw_abnormal_train_features', vid_name[:-9] + ".npy")

        # Remove last clip if number of frames is not equal to 16
        if frames % 16 != 0:
            clips = clips[:-1]

        prep_clips = [c3d_preprocess_input(np.array(clip)) for clip in clips]
        prep_clips = np.vstack(prep_clips)

        features = feature_extractor.predict(prep_clips)
        features = sklearn.preprocessing.normalize(features, axis=1)

        with open(feats_path, "wb") as f:
            np.save(f, features)
    count += 1


        
#test_videos = os.listdir(cfg.test_set)
#test_videos.sort()
#print("Processing test videos...")
#for vid_name in test_videos:
    #print("Processing {}".format(vid_name))
    #vid_path = os.path.join(cfg.test_set, vid_name)
    #feats_path = os.path.join(cfg.raw_test_features, vid_name[:-9] + ".npy")

    #clips, frames = video_util.get_video_clips(vid_path)

    # Remove last clip if number of frames is not equal to 16
    #if frames % 16 != 0:
        #clips = clips[:-1]

    #prep_clips = [c3dpreprocess_input(np.array(clip)) for clip in clips]
    #prep_clips = np.vstack(prep_clips)

    #features = feature_extractor.predict(prep_clips)
    #features = sklearn.preprocessing.normalize(features, axis=1)

    #with open(feats_path, "wb") as f:
        #np.save(f, features)

Processing Abnormal videos...
362 / 500 video: Stealing014_x264.mp4
363 / 500 video: Stealing015_x264.mp4
364 / 500 video: Stealing016_x264.mp4
365 / 500 video: Stealing017_x264.mp4
366 / 500 video: Stealing018_x264.mp4
367 / 500 video: Stealing019_x264.mp4
368 / 500 video: Stealing020_x264.mp4
369 / 500 video: Stealing021_x264.mp4
370 / 500 video: Stealing022_x264.mp4
371 / 500 video: Stealing023_x264.mp4
372 / 500 video: Stealing024_x264.mp4
373 / 500 video: Stealing025_x264.mp4
374 / 500 video: Stealing026_x264.mp4
375 / 500 video: Stealing027_x264.mp4
376 / 500 video: Stealing028_x264.mp4
377 / 500 video: Stealing029_x264.mp4
378 / 500 video: Stealing030_x264.mp4
379 / 500 video: Stealing031_x264.mp4
380 / 500 video: Stealing032_x264.mp4
381 / 500 video: Stealing035_x264.mp4
382 / 500 video: Stealing036_x264.mp4
383 / 500 video: Stealing037_x264.mp4
384 / 500 video: Stealing042_x264.mp4
385 / 500 video: Stealing043_x264.mp4
386 / 500 video: Stealing044_x264.mp4
387 / 500 video: Ste

# PREPROCESSING FEATURES

In [21]:
def transform_into_segments(features, n_segments=32):
    if features.shape[0] < n_segments:
        raise RuntimeError(
            "Number of prev segments lesser than expected output size"
        )

    cuts = np.linspace(0, features.shape[0], n_segments,
                       dtype=int, endpoint=False)

    new_feats = []
    for i, j in zip(cuts[:-1], cuts[1:]):
        new_feats.append(np.mean(features[i:j,:], axis=0))

    new_feats.append(np.mean(features[cuts[-1]:,:], axis=0))

    new_feats = np.array(new_feats)
    new_feats = sklearn.preprocessing.normalize(new_feats, axis=1)
    return new_feats

#for filename in os.listdir('./raw_normal_train_features'):
    #print("Processing {}".format(filename))
    #raw_file_path = os.path.join(
        #'./raw_normal_train_features', filename)
    #processed_file_path = os.path.join(
        #'./processed_normal_train_features', filename)

    #with open(raw_file_path, "rb") as f:
        #feats = np.load(f, allow_pickle=True)

    #try:
        #new_feats = transform_into_segments(feats)
        #with open(processed_file_path, "wb") as f:
            #np.save(f, new_feats, allow_pickle=True)
    #except RuntimeError:
        #print("Video {} too short".format(filename))

for filename in os.listdir('./raw_abnormal_train_features'):
    print("Processing {}".format(filename))
    raw_file_path = os.path.join(
        './raw_abnormal_train_features', filename)
    processed_file_path = os.path.join(
        './processed_abnormal_train_features', filename
    )
    with open(raw_file_path, "rb") as f:
        feats = np.load(f, allow_pickle=True)

    try:
        new_feats = transform_into_segments(feats)
        with open(processed_file_path, "wb") as f:
            np.save(f, new_feats, allow_pickle=True)
    except RuntimeError:
        print("Video {} too short".format(filename))

#for filename in os.listdir(cfg.raw_test_features):
    #print("Processing {}".format(filename))
    #raw_file_path = os.path.join(
        #cfg.raw_test_features, filename
    #)
    #processed_file_path = os.path.join(
        #cfg.processed_test_features, filename
    #)
    #with open(raw_file_path, "rb") as f:
        #feats = np.load(f, allow_pickle=True)

    #try:
        #new_feats = transform_into_segments(feats)
        #with open(processed_file_path, "wb") as f:
            #np.save(f, new_feats, allow_pickle=True)
    #except RuntimeError:
        #print("Video {} too short".format(filename))

Processing Robbery150.npy
Processing Stealing021.npy
Processing Robbery135.npy
Processing Robbery102.npy
Processing Vandalism023.npy
Processing Robbery035.npy
Processing Stealing104.npy
Processing Stealing059.npy
Processing Robbery009.npy
Processing RoadAccidents028.npy
Processing Stealing003.npy
Processing Robbery139.npy
Processing RoadAccidents071.npy
Processing Robbery124.npy
Processing Robbery006.npy
Processing Robbery080.npy
Processing Robbery012.npy
Processing Robbery085.npy
Processing Robbery138.npy
Processing Vandalism020.npy
Processing RoadAccidents055.npy
Processing RoadAccidents143.npy
Processing Robbery088.npy
Processing RoadAccidents002.npy
Video RoadAccidents002.npy too short
Processing Shooting018.npy
Processing Robbery128.npy
Processing RoadAccidents022.npy
Processing Vandalism046.npy
Processing Robbery067.npy
Processing RoadAccidents019.npy
Processing Shooting005.npy
Processing RoadAccidents151.npy
Processing Shooting006.npy
Processing RoadAccidents081.npy
Processing R

In [22]:
import os

print(len(os.listdir('./processed_abnormal_train_features')))
print(len(os.listdir('./processed_normal_train_features')))
#print(len(os.listdir('./Training-Normal-Videos-Part-2')))

843
454
